In [ ]:
! pip install -q langchain_community langchainhub chromadb langchain transformers sentence-transformers langchain-experimental langchain-huggingface langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.3 MB/s eta 0:00:00

`(2) LangSmith`

https://docs.smith.langchain.com/

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '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'
os.environ['LANGSMITH_PROJECT']="1st demo"

In [ ]:
from google.colab import userdata
from huggingface_hub import login
import os

token = userdata.get("HF_TOKEN")
if not token:
    raise RuntimeError("Chưa thấy secret HF_TOKEN trong Colab Secrets")

# Đăng nhập HF Hub (lưu vào cache ~/.cache/huggingface)
login(token)

# (tuỳ chọn) set env var để LangChain wrappers đọc
os.environ["HUGGINGFACEHUB_API_TOKEN"] = token

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


In [ ]:
import requests
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

## Blog loading ##

In [ ]:
!pip -q install tiktoken

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [ ]:
print(docs)

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

## Chunking and Embedding ##

In [ ]:
from typing import List
from langchain.schema import Document

In [ ]:
def semantic_split(docs: List[Document]) -> List[Document]:
    """
    Semantic chunking

    Args:
        docs (List[Document]): List of documents to chunk

    Returns:
        List[Document]: List of chunked documents
    """
    splitter = SemanticChunker(
        GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
        buffer_size = 2, breakpoint_threshold_type="percentile",
        breakpoint_threshold_amount=95,
    )
    return splitter.split_documents(docs)

In [ ]:
chunks = semantic_split(docs)

In [ ]:
print(chunks)

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver. Agent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks. Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes 

## vectorstore storing ##

In [ ]:
!pip install rank-bm25

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_community.retrievers import BM25Retriever

In [ ]:
# Index
emb_llm = GoogleGenerativeAIEmbeddings(model="models/embedding-001") #<-- embeding LLM
vectorstore = Chroma.from_documents(documents=chunks,
                                    embedding=emb_llm)

dense_retriever = vectorstore.as_retriever(search_type="mmr",
    search_kwargs={"k": 10, "fetch_k": 40, "lambda_mult": 0.5})
# Lexical retriever (BM25)
bm25 = BM25Retriever.from_documents(chunks)
bm25.k = 10

## applying PRF ##

In [ ]:
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def _extract_prf_terms(docs, top_m=6):
    """Lấy top_m term để mở rộng query từ các doc phản hồi giả định (PRF)."""
    texts = [d.page_content for d in docs if getattr(d, "page_content", None)]
    if not texts:
        return []
    vec = TfidfVectorizer(ngram_range=(1,2), stop_words="english", max_features=4096)
    X = vec.fit_transform(texts)                # shape: (#docs, #terms)
    scores = np.asarray(X.sum(axis=0)).ravel()  # tổng tf-idf theo cột (term)
    terms = np.array(vec.get_feature_names_out())
    idx = scores.argsort()[::-1][:top_m]
    return terms[idx].tolist()

In [ ]:
def _mean_vector(vectors):
    # vectors: List[List[float]]
    if not vectors:
        return None
    return [float(sum(col))/len(vectors) for col in zip(*vectors)]

def _dedup(docs):
    seen, out = set(), []
    for d in docs:
        key = (d.metadata.get("source") or d.metadata.get("url"), d.page_content[:200])
        if key in seen:
            continue
        seen.add(key)
        out.append(d)
    return out

In [ ]:
def hybrid_retrieve_prf(
    query: str,
    *,
    k: int = 10,           # số doc trả về cuối cùng mỗi nhánh
    fb_docs: int = 8,      # số doc dùng làm phản hồi giả định (feedback set)
    alpha: float = 0.6,    # trọng số pha trộn vector: prf_vec = (1-alpha)*q + alpha*mean(docs)
    prf_terms: int = 6     # số term thêm vào query cho BM25
):
    # Lượt 1: baseline
    dense_1 = dense_retriever.invoke(query)
    sparse_1 = bm25.invoke(query)
    # Chọn tập phản hồi giả định (kết hợp 2 nhánh)
    fb = []
    fb += dense_1[:max(1, fb_docs // 2)]
    fb += sparse_1[:max(1, fb_docs - len(fb))]
    if not fb:
        # fallback: không có doc nào → trả về kết quả baseline
        return [dense_1[:k], sparse_1[:k]]
    # --- Sparse PRF: mở rộng query bằng terms từ fb ---
    expansion = _extract_prf_terms(fb, top_m=prf_terms)
    expanded_query = query if not expansion else (query + " " + " ".join(expansion))
    # --- Dense PRF: pha trộn embedding query với trung bình embedding từ fb ---
    fb_texts = [d.page_content for d in fb if getattr(d, "page_content", None)]
    try:
        q_vec = emb_llm.embed_query(query)
        if fb_texts:
            fb_vecs = emb_llm.embed_documents(fb_texts)
            m_vec = _mean_vector(fb_vecs)
            prf_vec = [(1 - alpha) * q + alpha * m for q, m in zip(q_vec, m_vec)]
            dense_2 = vectorstore.similarity_search_by_vector(prf_vec, k=k)
        else:
            dense_2 = dense_1[:k]
    except Exception:
        # Nếu embedding provider lỗi, dùng lại kết quả baseline
        dense_2 = dense_1[:k]
    # Lượt 2 BM25 với query mở rộng
    sparse_2 = bm25.invoke(expanded_query)[:k]
    # Khử trùng lặp nhẹ trước khi trả
    return [_dedup(dense_2)[:k], _dedup(sparse_2)[:k]]

### Prompt

In [ ]:
from langchain.prompts import ChatPromptTemplate
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import ChatHuggingFace
from langchain_huggingface.llms import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
# Multi Query: Different Perspectives
template = """bạn là 1 trợ lý AI thông minh. Hãy tạo cho tôi 3 phiên bản câu hỏi khác từ câu hỏi gốc được gửi bởi người dùng để truy hồi các tài liệu liên quan được lưu trong vector database.
              Bằng cách tạo nhiều góc nhìn (perspective) khác nhau từ câu hỏi gốc của người dùng, mục tiêu của bạn là giúp người dùng vượt qua những giới hạn về distance-based similarity search.
              Chỉ in ra 3 câu truy vấn mới: không đánh số; không ngoặc kép; mỗi truy vấn 1 dòng, trả lời bằng tiếng Anh.
              câu hỏi gốc: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_google_genai import ChatGoogleGenerativeAI

chat_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2)

generate_queries = (
    prompt_perspectives
    | chat_llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))    #(lambda x: [s for s in (t.strip() for t in x.split("\n")) if s])
)

## re-rank ##

In [ ]:
import json
from collections import defaultdict
from langchain.schema import Document

### rrf function ###

In [ ]:
def reciprocal_rank_fusion(results, k=60, return_scores=False, key_fn=None):
    """
    RRF gộp nhiều list xếp hạng (list[list[Document]]).
    Mặc định trả về list[Document] (đã rerank & dedup).
    Nếu return_scores=True, trả về list[(Document, score)].
    """

    # Hàm tạo khóa dedup
    if key_fn is None:
        def key_fn(doc):
            if isinstance(doc, Document):
                src = doc.metadata.get("source", "")
                page = doc.metadata.get("page", "")
                # khóa ổn định theo nội dung + vài trường nhận diện
                return json.dumps([doc.page_content, src, page], ensure_ascii=False)
            try:
                return json.dumps(doc, ensure_ascii=False, sort_keys=True)
            except TypeError:
                return str(doc)

    fused_scores = defaultdict(float)
    keep_doc = {}

    for docs in results:                      # results: list of ranked lists
        for rank, doc in enumerate(docs):
            key = key_fn(doc)
            keep_doc[key] = doc               # lưu doc tương ứng với key
            fused_scores[key] += 1.0 / (rank + k)

    # sắp xếp theo fused score giảm dần
    items = sorted(fused_scores.items(), key=lambda kv: kv[1], reverse=True)

    if return_scores:
        return [(keep_doc[k], score) for k, score in items]
    else:
        return [keep_doc[k] for k, _ in items]

In [ ]:
def retrieve_rrf_for_queries(queries: list[str]):
    per_query_rrf = []
    for q in queries:
        lists = hybrid_retrieve_prf(q)  #hybrid_retrieve(q)
        per_query_rrf.append(reciprocal_rank_fusion(lists, k=60))
    return reciprocal_rank_fusion(per_query_rrf, k=60)

### CrossEncoder ###

In [ ]:
!pip -q install sentence-transformers

In [ ]:
from sentence_transformers import CrossEncoder
ce = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

def cross_rerank(query: str, docs: list[Document], top_k=8):
    pairs = [(query, d.page_content) for d in docs]
    scores = ce.predict(pairs)
    ranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    return [d for d, _ in ranked[:top_k]]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [ ]:
retrieval_chain = generate_queries | (lambda qs: retrieve_rrf_for_queries(qs))
retrieved_docs = retrieval_chain.invoke({"question":question})
#docs = cross_rerank(question, retrieved_docs, top_k=8)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

In [ ]:
def join_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

In [ ]:
from langchain_core.runnables import RunnableLambda

##Answer Generation##


In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1)

final_rag_chain = (
    {
        "question": itemgetter("question"),
        "retrieved_docs": retrieval_chain,
    }
    | RunnableLambda(lambda x: cross_rerank(x["question"], x["retrieved_docs"], top_k=8))
    | RunnableLambda(join_docs)
    | (lambda context: {"context": context, "question": question})
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = "What is task decomposition?"

In [ ]:
final_rag_chain.invoke({"question":question})

'Task decomposition is the process by which an autonomous agent breaks down large, complicated tasks into smaller, simpler, and more manageable subgoals or steps. This enables the agent to plan ahead and handle complex tasks efficiently.\n\nMethods for task decomposition mentioned in the context include:\n*   **Chain of Thought (CoT)**: A prompting technique that instructs the model to "think step by step" to decompose hard tasks into smaller, simpler steps.\n*   **Tree of Thoughts**: Extends CoT by exploring multiple reasoning possibilities at each step, generating multiple thoughts per step to create a tree structure.\n*   **LLM with simple prompting**: Using prompts like "Steps for XYZ.\\n1." or "What are the subgoals for achieving XYZ?".\n*   **Task-specific instructions**: Providing specific instructions, such as "Write a story outline." for writing a novel.\n*   **Human inputs**: Incorporating human guidance for decomposition.\n*   **LLM+P**: Relying on an external classical plan

## for streaming ##


In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.1)

final_rag_chain = (
    {
        "question": itemgetter("question"),
        "retrieved_docs": retrieval_chain,
    }
    | RunnableLambda(lambda x: {
              "question": x["question"],
              "context_docs": cross_rerank(x["question"], x["retrieved_docs"], top_k=8)})
    | RunnableLambda(lambda x: {
        "question": x["question"],
        "context": join_docs(x["context_docs"])})
    | (lambda context: {"context": context, "question": question})
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
import os
import json
from google.colab import userdata
from pyngrok import ngrok
from flask import Flask, jsonify, request
from flask_cors import CORS

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

# Khởi tạo Flask app
app = Flask(__name__)
CORS(app)

# NGROK setup
authtoken = "YOUR_NGROK_AUTHTOKEN"  # hoặc lấy từ userdata / biến môi trường
ngrok.set_auth_token(authtoken)


@app.route("/v1/chat", methods=["POST"])
def chat_v1():
    data = request.get_json()

    # 👇 Từ streamlit gửi lên
    raw_message = data.get("message", {})
    question = raw_message.get("human", "")
    context = data.get("context", [])
    stream = data.get("stream", True)

    print(">> Received Question:", question)
    print(">> Context len:", len(context))

    try:
        if stream:
            def generate():
                for chunk in final_rag_chain.stream({"question": question}):
                    yield chunk
            return app.response_class(generate(), mimetype="text/plain")
        else:
            result = final_rag_chain.invoke({"question": question})
            return jsonify({"response": result})
    except Exception as e:
        print("[ERROR]", str(e))
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    public_url = ngrok.connect(5000)
    print(f" * Public URL: {public_url}")
    app.run(port=5000)
